<a href="https://colab.research.google.com/github/EmperorBlackMD/AIHC-5010/blob/main/FL_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Loading my file
import pandas as pd

path = '/content/drive/MyDrive/FL_project/FL_data.xlsx'
df = pd.read_excel(path)
df.head()

,MER ERA,Mayo Clinic Number,Clinic ID,MER ID,LEO ID,Age @ Diagnosis,Gender,Race,Ethnicity,Diagnosis\nBiopsy Date,...,Tumor Size 1,Funky Dx,Tissue Status,Tumor % 2,Tumor Size 2,Tumor % 3,Tumor Size 3,Tumor % 4,Tumor Size 4,Accession Available
0,MER 2.0 (07/01/2015 - 07/31/2021),4853674.0,4853674,NaN,MA117104,78,Male,White,Not Hispanic or Latino,2018-06-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accession # Available
1,MER 2.0 (07/01/2015 - 07/31/2021),7551765.0,7551765,NaN,MA116538,77,Male,White,Not Hispanic or Latino,2018-03-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accession # Available
2,MER 2.0 (07/01/2015 - 07/31/2021),7711429.0,7711429,NaN,MA117068,77,Male,White,Not Hispanic or Latino,2018-02-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accession # Available
3,MER 1.0 (2002- 06/30/2015),1018649.0,1018649,MC1689,NaN,67,Female,White,Not Hispanic or Latino,2005-08-22,...,2.0,NaN,Tissue exhaused,NaN,NaN,NaN,NaN,NaN,NaN,Accession # Available
4,MER 2.0 (07/01/2015 - 07/31/2021),10258704.0,10258704,NaN,MC119987,65,Male,White,Not Hispanic or Latino,2019-09-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accession # Available


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Column mapping
COLS = {
    "id": "Clinic ID",
    "age": "Age @ Diagnosis",
    "sex": "Gender",
    "race": "Race",
    "dx_date": "Diagnosis Biopsy Date",
    "histology": "Biopsy Histology",
    "ht": "Transformation",
    "ht_date": "Date of transformation",
    "pod24": "EFS24",
    "vital": "Vital Status",
    "fu_months": "FU Months",
    "efs_status": "EFS Status",
    "efs_months": "EFS months",
    "cod": "Primary COD",
    "bulky": "Bulky Disease",
    "b_symptoms": "B Symptoms",
    "stage": "Ann Arbor Stage",
    "amc": "AMC",
    "alc": "ALC",
    "anc": "ANC",
    "bili": "Bilirubin (mg dL)",
    "wbc": "WBC (x109 L o K mm3)",
    "plt": "PLT (x109 L)",
    "hgb": "HGB (g dL)",
    "alb": "Serum Albumin (g dL)",
    "cr": "Creatinine (mg dL)",
    "age_group": "Age Group",
    "stage_group": "Ann Arbor Stage Group",
    "ps_group": "PS Group",
    "ldh_group": "LDH Group",
    "ex_sites": "Num of Ex. Sites",
    "flipi": "FLIPI",
    "flipi_group": "FLIPI Group",
    "immunochemo": "FL Immuno-chemo",
    "bm_group": "BM Involvement Group",
    "multi_hist": "Multiple FL Histologies",
    "fl_grade": "FL Grade",
}

In [7]:
# Ensure ID uniqueness
n = len(df)
n_unique = df[COLS['id']].nunique(dropna=False)
print('Rows:', n, '| Unique IDs:', n_unique)

Rows: 1987 | Unique IDs: 1987


In [22]:
# Parse dates
for dcol in [COLS['dx_date'], COLS['ht_date']]:
    if dcol in df.columns:
        df[dcol] = pd.to_datetime(df[dcol], errors='coerce')

# Normalize common binary/categorical encodings
def map_yes_no(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if s in ["yes", "y", "1", "true", "t"]: return 1
    if s in ["no", "n", "0", "false", "f"]: return 0
    return np.nan

# Histologic transformation
if COLS["ht"] in df.columns:
    df["_ht01"] = df[COLS["ht"]].map(map_yes_no)

# Bulky, B symptoms, immunochemo if yes/no
for c in ["bulky", "b_symptoms", "immunochemo"]:
    col = COLS.get(c)
    if col in df.columns:
        df[f"_{c}01"] = df[col].map(map_yes_no)

# Vital status
if COLS["vital"] in df.columns:
    def map_vital(x):
        if pd.isna(x): return np.nan
        s = str(x).strip().lower()
        if "Deceased" in s or "dead" in s: return 1
        if "Alive" in s: return 0
        return np.nan
    df["_dead01"] = df[COLS["vital"]].map(map_vital)

# EFS status
if COLS["efs_status"] in df.columns:
    def map_event(x):
        if pd.isna(x): return np.nan
        s = str(x).strip().lower()
        if "event" in s: return 1
        if "no event" in s or "No event" in s: return 0
        return np.nan
    df["_efs_event01"] = df[COLS["efs_status"]].map(map_event)

# EFS24 (<24m are blank) into a numeric binary
def parse_efs24_to_bin(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s == "":
        return np.nan
    if "fail" in s:
        return 1
    if "achiev" in s:
        return 0
    return np.nan  # unexpected category

df["EFS24_bin"] = df["EFS24"].apply(parse_efs24_to_bin)

# -- Numeric coercion for labs and key numeric fields ----------
numeric_cols = [
    COLS["age"], COLS["amc"], COLS["alc"], COLS["anc"], COLS["bili"],
    COLS["wbc"], COLS["plt"], COLS["hgb"], COLS["alb"], COLS["cr"],
    COLS["fu_months"], COLS["efs_months"], COLS["ex_sites"], COLS["flipi"]
]
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [23]:
# Missingness summary (focused columns) ----------
focus_cols = list(COLS.values())
focus_cols = [c for c in focus_cols if c in df.columns]
miss = df[focus_cols].isna().mean().sort_values(ascending=False)
print("\nMissingness (fraction):\n", miss.head(25))


Missingness (fraction):
 Date of transformation    0.910418
ANC                       0.411173
AMC                       0.315048
ALC                       0.191746
LDH Group                 0.173125
EFS24                     0.122798
BM Involvement Group      0.085556
Bulky Disease             0.075994
Ann Arbor Stage           0.051334
Ann Arbor Stage Group     0.051334
Num of Ex. Sites          0.045798
PS Group                  0.036236
FL Immuno-chemo           0.017111
B Symptoms                0.004529
Primary COD               0.003523
FL Grade                  0.001510
Clinic ID                 0.000000
Age @ Diagnosis           0.000000
Gender                    0.000000
Race                      0.000000
FU Months                 0.000000
EFS Status                0.000000
Transformation            0.000000
Vital Status              0.000000
Age Group                 0.000000
dtype: float64


In [37]:
missing_vars = {
    "ANC": "ANC",
    "AMC": "AMC",
    "ALC": "ALC",
    "LDH_group": "LDH Group"
}

for k, col in missing_vars.items():
    if col in df.columns:
     df[f"miss_{k}"] = df[col].isna().astype(int)

# Quick prevalence check
for k in missing_vars:
    print(k, df[f"miss_{k}"].mean())


ANC 0.4111726220432813
AMC 0.31504781077000504
ALC 0.19174635128334172
LDH_group 0.1731253145445395


In [38]:
# Little's test to check MCAR
from scipy import stats
import numpy as np

def little_mcar_test(data):
    """
    Little's MCAR test.
    data: pandas DataFrame with only variables to test (numeric preferred).
    """
    X = data.copy()

    # Drop rows with all missing
    X = X.dropna(how="all")

    # Mean vector
    mean_vec = X.mean(skipna=True).values

    # Covariance matrix
    cov = X.cov()

    test_stat = 0
    df_total = 0

    for pattern, group in X.groupby(X.isna().apply(tuple, axis=1)):
        observed = ~np.array(pattern)
        group_obs = group.loc[:, observed]

        if group_obs.shape[0] <= 1:
            continue

        mean_diff = group_obs.mean().values - mean_vec[observed]
        cov_obs = cov.loc[group_obs.columns, group_obs.columns].values

        try:
            inv_cov = np.linalg.inv(cov_obs)
        except np.linalg.LinAlgError:
            continue

        test_stat += group_obs.shape[0] * mean_diff.T @ inv_cov @ mean_diff
        df_total += observed.sum()

    p_value = 1 - stats.chi2.cdf(test_stat, df_total)
    return test_stat, df_total, p_value

labs_for_mcar = df[
    ["ANC", "AMC", "ALC"]
].copy()

stat, dof, p = little_mcar_test(labs_for_mcar)

print(f"Little's MCAR test: χ²={stat:.2f}, df={dof}, p={p:.4g}")


Little's MCAR test: χ²=7.92, df=7, p=0.3399


In [42]:
# cHECK IF MISSINGNESS IS ASSOCIATEED WITH BASELINE CLINICAL VARIABLES
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

predictors = [
    "Age @ Diagnosis",
    "Ann Arbor Stage Group",
    "FLIPI Group",
    "Bulky Disease",
    "B Symptoms",
    "EFS24_bin"
]

if 'EFS24_bin' in df.columns:
    df['EFS24_bin'] = pd.to_numeric(df['EFS24_bin'], errors='coerce')
# Prepare X with dummies and force all columns to numeric
Xraw = df[predictors].copy()

# Convert obvious numeric columns
if "Age @ Diagnosis" in Xraw.columns:
    Xraw["Age @ Diagnosis"] = pd.to_numeric(Xraw["Age @ Diagnosis"], errors="coerce")

# Convert all categorical to string consistently (prevents mixed-type dummies)
cat_cols = [c for c in Xraw.columns if c != "Age @ Diagnosis"]
for c in cat_cols:
    Xraw[c] = Xraw[c].astype("string")

Xdum = pd.get_dummies(Xraw, drop_first=True)

# Force everything to numeric (key fix)
Xdum = Xdum.apply(pd.to_numeric, errors="coerce")

# Add constant AFTER numeric conversion
Xdum = sm.add_constant(Xdum, has_constant="add")

# Fit a missingness model for each variable
for k in missing_vars.keys():
    miss_col = f"miss_{k}"
    if miss_col not in df.columns:
        continue

    y = df[miss_col]
    y = pd.to_numeric(y, errors="coerce")

    # Combine and drop rows with any NA in X or y
    tmp = pd.concat([y.rename("y"), Xdum], axis=1).dropna()
    y_clean = tmp["y"].astype(int)
    X_clean = tmp.drop(columns=["y"]).astype(float)

    # Skip degenerate cases
    if y_clean.nunique() < 2:
        print(f"\nMissingness model for {k}: skipped (outcome has <2 classes after dropping NAs)")
        continue

    try:
        model = sm.Logit(y_clean, X_clean).fit(disp=False)
        print(f"\nMissingness model for {k} (miss_{k}=1 means {k} is missing)")
        out = model.summary2().tables[1][["Coef.", "Std.Err.", "P>|z|"]].sort_values("P>|z|")
        print(out.head(15))
    except Exception as e:
        print(f"\nMissingness model for {k}: FAILED with error: {e}")


Missingness model for ANC (miss_ANC=1 means ANC is missing)
                                 Coef.  Std.Err.     P>|z|
Age @ Diagnosis              -0.017925  0.003856  0.000003
B Symptoms_Unconfirmed        1.445231  0.332891  0.000014
EFS24_bin_1.0                 0.296445  0.109527  0.006798
const                         0.653889  0.245315  0.007687
Bulky Disease_Yes            -0.339462  0.166528  0.041503
B Symptoms_Yes               -0.180683  0.145146  0.213194
FLIPI Group_2 Intermediate    0.024073  0.131080  0.854287
Ann Arbor Stage Group_III-IV  0.009808  0.127465  0.938664
FLIPI Group_3 High            0.000150  0.159972  0.999252

Missingness model for AMC (miss_AMC=1 means AMC is missing)
                                 Coef.  Std.Err.         P>|z|
B Symptoms_Unconfirmed        1.775673  0.333367  1.001362e-07
Age @ Diagnosis              -0.016779  0.004062  3.614323e-05
B Symptoms_Yes               -0.408556  0.163283  1.234490e-02
Bulky Disease_Yes            -0.2997

#ANC missingness (41% missing) — clearly NOT MCAR

Key predictors:

Age @ Diagnosis (p = 3e-6):
Older patients are less likely to have ANC missing (negative coefficient).

B Symptoms = Unconfirmed (p = 1e-5):
Strong association → documentation/workup intensity effect.

EFS24 failure (p = 0.007):
Missingness differs by outcome → very strong evidence against MCAR.

Bulky Disease (p = 0.04):
Suggests disease burden/workup differences.

Conclusion: ANC missingness is MAR, possibly outcome-informative.

#AMC missingness (32% missing) — NOT MCAR

Key predictors:

B Symptoms = Unconfirmed (p ≈ 1e-7)

Age @ Diagnosis (p ≈ 4e-5)

B Symptoms = Yes (p = 0.012)

Outcome (EFS24) is not significant here, but baseline predictors are.

Conclusion: AMC missingness is MAR, driven by clinical documentation/workup patterns.

#ALC missingness (19% missing) — NOT MCAR

Key predictors:

B Symptoms = Unconfirmed (p ≈ 6e-13)

FLIPI Group (Intermediate/High) (p = 0.046 / 0.001)

This is important: prognostic risk itself predicts missingness, which is classic MAR.

Conclusion: ALC missingness is MAR, tied to disease risk.

# LDH group missingness (17% missing) — strongly NOT MCAR

Key predictors:

B Symptoms = Unconfirmed (p ≈ 1e-13)

FLIPI Group (Intermediate/High) (p ≈ 1e-4 / 3e-10)

This is textbook:

Low-risk patients often lack LDH documentation

High-risk patients almost always have LDH measured

Conclusion: LDH missingness is MAR, and this is extremely common in lymphoma datasets.


##Missingness was assessed using logistic regression models predicting missingness from baseline clinical variables. Missingness for ANC, AMC, ALC, and LDH was significantly associated with observed covariates, including age, B symptoms, FLIPI risk group, and early event status, indicating that data were not missing completely at random (MCAR) but were consistent with a missing at random (MAR) mechanism.

In [44]:
for v in ["ANC", "AMC", "ALC", "LDH Group"]:
    df[f"{v}_missing"] = df[v].isna().astype(int)


In [45]:
# 2) Evaluable cohort and event rate
evaluable = df[df["EFS24_bin"].isin([0, 1])].copy()
print("EFS24 evaluable n:", len(evaluable))
print("EFS24 failure rate:", evaluable["EFS24_bin"].mean())

# 3) Debug: show what values were present
print("\nRaw EFS24 value counts:")
print(df["EFS24"].value_counts(dropna=False))

print("\nParsed EFS24_bin value counts:")
print(df["EFS24_bin"].value_counts(dropna=False))

EFS24 evaluable n: 1743
EFS24 failure rate: 0.27309236947791166

Raw EFS24 value counts:
EFS24
Achieved    1267
Failed       476
NaN          244
Name: count, dtype: int64

Parsed EFS24_bin value counts:
EFS24_bin
0.0    1267
1.0     476
NaN     244
Name: count, dtype: int64


In [46]:
# ---------- Baseline benchmark model for EFS24 (logistic regression) ----------
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV

# Numeric and categorical predictors (conservative initially)
features_num = [
    COLS["age"], COLS["amc"], COLS["alc"], COLS["anc"], COLS["bili"],
    COLS["wbc"], COLS["plt"], COLS["hgb"], COLS["alb"], COLS["cr"],
    COLS["flipi"], COLS["ex_sites"],
]
features_cat = [
    COLS["sex"], COLS["race"], COLS["stage"], COLS["stage_group"],
    COLS["ps_group"], COLS["ldh_group"], COLS["flipi_group"],
    COLS["bm_group"], COLS["fl_grade"], COLS["bulky"], COLS["b_symptoms"],
]

# Keep only columns that exist
features_num = [c for c in features_num if c in evaluable.columns]
features_cat = [c for c in features_cat if c in evaluable.columns]

X = evaluable[features_num + features_cat].copy()
y = evaluable["EFS24_bin"].astype(int).values

print("\nUsing numeric features:", features_num)
print("Using categorical features:", features_cat)


Using numeric features: ['Age @ Diagnosis', 'AMC', 'ALC', 'ANC', 'FLIPI', 'Num of Ex. Sites']
Using categorical features: ['Gender', 'Race', 'Ann Arbor Stage', 'Ann Arbor Stage Group', 'PS Group', 'LDH Group', 'FLIPI Group', 'BM Involvement Group', 'FL Grade', 'Bulky Disease', 'B Symptoms']


In [49]:
# Preprocessing pipeline (impute + scale numeric; impute + one-hot categorical)
pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc", StandardScaler())
        ]), features_num),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore"))
        ]), features_cat),
    ],
    remainder="drop"
)

#  4) Model 1: Regularized logistic regression baseline (elastic net)
# -------------------------
base_clf = LogisticRegression(
    penalty="elasticnet",
    solver="saga",
    l1_ratio=0.2,      # adjust later in tuning
    C=1.0,             # adjust later in tuning
    max_iter=8000,
    class_weight=None  # consider "balanced" only if needed
)

pipe = Pipeline([
    ("pre", pre),
    ("clf", base_clf)
])

# -------------------------
# 5) Cross-validated performance (AUROC, AUPRC, Brier)
# -------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

aucs, aprs, briers = [], [], []
oof_pred = np.zeros_like(y, dtype=float)

for fold, (tr, te) in enumerate(cv.split(X, y), start=1):
    pipe.fit(X.iloc[tr], y[tr])
    p = pipe.predict_proba(X.iloc[te])[:, 1]
    oof_pred[te] = p

    aucs.append(roc_auc_score(y[te], p))
    aprs.append(average_precision_score(y[te], p))
    briers.append(brier_score_loss(y[te], p))

    print(f"Fold {fold}: AUROC={aucs[-1]:.3f} | AUPRC={aprs[-1]:.3f} | Brier={briers[-1]:.3f}")

print("\nElasticNet LogReg (5-fold CV)")
print("AUROC: mean {:.3f} (sd {:.3f})".format(np.mean(aucs), np.std(aucs)))
print("AUPRC: mean {:.3f} (sd {:.3f})".format(np.mean(aprs), np.std(aprs)))
print("Brier: mean {:.3f} (sd {:.3f})".format(np.mean(briers), np.std(briers)))

Fold 1: AUROC=0.520 | AUPRC=0.298 | Brier=0.208
Fold 2: AUROC=0.613 | AUPRC=0.425 | Brier=0.189
Fold 3: AUROC=0.596 | AUPRC=0.375 | Brier=0.195
Fold 4: AUROC=0.689 | AUPRC=0.499 | Brier=0.180
Fold 5: AUROC=0.560 | AUPRC=0.327 | Brier=0.201

ElasticNet LogReg (5-fold CV)
AUROC: mean 0.595 (sd 0.057)
AUPRC: mean 0.385 (sd 0.072)
Brier: mean 0.195 (sd 0.010)


In [50]:
# Optional: Probability calibration (nested CV style, lightweight)
#    Use this if you plan to report calibrated risks clinically.
# -------------------------
# Note: CalibratedClassifierCV performs internal CV; keep it simple initially.
calibrated_pipe = Pipeline([
    ("pre", pre),
    ("cal", CalibratedClassifierCV(
        estimator=base_clf,
        method="isotonic",  # or "sigmoid" for smaller samples
        cv=3
    ))
])

cal_aucs, cal_aprs, cal_briers = [], [], []
for fold, (tr, te) in enumerate(cv.split(X, y), start=1):
    calibrated_pipe.fit(X.iloc[tr], y[tr])
    p = calibrated_pipe.predict_proba(X.iloc[te])[:, 1]
    cal_aucs.append(roc_auc_score(y[te], p))
    cal_aprs.append(average_precision_score(y[te], p))
    cal_briers.append(brier_score_loss(y[te], p))

print("\nCalibrated ElasticNet LogReg (isotonic, 5-fold outer CV)")
print("AUROC: mean {:.3f} (sd {:.3f})".format(np.mean(cal_aucs), np.std(cal_aucs)))
print("AUPRC: mean {:.3f} (sd {:.3f})".format(np.mean(cal_aprs), np.std(cal_aprs)))
print("Brier: mean {:.3f} (sd {:.3f})".format(np.mean(cal_briers), np.std(cal_briers)))


Calibrated ElasticNet LogReg (isotonic, 5-fold outer CV)
AUROC: mean 0.593 (sd 0.059)
AUPRC: mean 0.372 (sd 0.068)
Brier: mean 0.194 (sd 0.008)


In [51]:
# (Optional) Simple threshold table for clinical-style cut points
# -------------------------
def threshold_metrics(y_true, p_pred, thr):
    y_hat = (p_pred >= thr).astype(int)
    tp = np.sum((y_true == 1) & (y_hat == 1))
    fp = np.sum((y_true == 0) & (y_hat == 1))
    tn = np.sum((y_true == 0) & (y_hat == 0))
    fn = np.sum((y_true == 1) & (y_hat == 0))
    sens = tp / (tp + fn) if (tp + fn) else np.nan
    spec = tn / (tn + fp) if (tn + fp) else np.nan
    ppv  = tp / (tp + fp) if (tp + fp) else np.nan
    npv  = tn / (tn + fn) if (tn + fn) else np.nan
    return {"thr": thr, "sens": sens, "spec": spec, "ppv": ppv, "npv": npv}

print("\nThreshold metrics using out-of-fold predictions (uncalibrated baseline):")
for thr in [0.1, 0.2, 0.3, 0.4, 0.5]:
    m = threshold_metrics(y, oof_pred, thr)
    print("thr={thr:.1f} | sens={sens:.2f} spec={spec:.2f} ppv={ppv:.2f} npv={npv:.2f}".format(**m))


Threshold metrics using out-of-fold predictions (uncalibrated baseline):
thr=0.1 | sens=0.99 spec=0.01 ppv=0.27 npv=0.86
thr=0.2 | sens=0.80 spec=0.28 ppv=0.30 npv=0.79
thr=0.3 | sens=0.42 spec=0.68 ppv=0.33 npv=0.76
thr=0.4 | sens=0.18 spec=0.91 ppv=0.44 npv=0.75
thr=0.5 | sens=0.06 spec=0.97 ppv=0.45 npv=0.73


# An updated model implementing missing values

In [57]:
# =========================
# EFS24 baseline benchmark modeling with MAR-aware missingness handling
#   - EFS24 coded: Achieved / Failed / blank
#   - Add missingness indicators for ANC/AMC/ALC/LDH Group
#   - 5-fold stratified CV with AUROC, AUPRC, Brier
# =========================

# 2) Create missingness indicators (MAR-aware)
# -------------------------
missing_targets = ["anc", "amc", "alc", "ldh_group"]
for key in missing_targets:
    col = COLS[key]
    if col not in evaluable.columns:
        raise KeyError(f"Missing expected column for missingness indicator: '{col}'")
    evaluable[f"{col}__missing"] = evaluable[col].isna().astype(int)

# Optionally coerce numeric labs/age to numeric
numeric_to_coerce = [COLS["age"], COLS["anc"], COLS["amc"], COLS["alc"],
                     COLS.get("wbc"), COLS.get("plt"), COLS.get("hgb"),
                     COLS.get("alb"), COLS.get("cr"), COLS.get("bili"),
                     COLS.get("ex_sites"), COLS.get("flipi")]
numeric_to_coerce = [c for c in numeric_to_coerce if c is not None and c in evaluable.columns]
for c in numeric_to_coerce:
    evaluable[c] = pd.to_numeric(evaluable[c], errors="coerce")

#  Define feature sets
#    - We'll run two models:
#        A) clinical-only WITHOUT ANC/AMC/ALC/LDH_group
#        B) clinical+labs WITH ANC/AMC/ALC/LDH_group + missing indicators
# -------------------------

# Base clinical predictors (no labs)
features_num_base = [
    COLS["age"],
    COLS["flipi"],      # keep if available and numeric
    COLS["ex_sites"],   # keep if available and numeric
]
features_cat_base = [
    COLS["sex"], COLS["race"], COLS["stage"], COLS["stage_group"],
    COLS["ps_group"], COLS["flipi_group"], COLS["fl_grade"],
    COLS["bm_group"], COLS["bulky"], COLS["b_symptoms"],
]

# Labs to include in the expanded model
features_num_labs = [COLS["anc"], COLS["amc"], COLS["alc"]]
features_cat_labs = [COLS["ldh_group"]]

# Add missingness indicators (these are numeric 0/1)
missing_indicator_cols = [f"{COLS[k]}__missing" for k in missing_targets]

# Keep only existing columns
def keep_existing(cols, df_):
    return [c for c in cols if c in df_.columns]

features_num_base = keep_existing(features_num_base, evaluable)
features_cat_base = keep_existing(features_cat_base, evaluable)

features_num_labs = keep_existing(features_num_labs, evaluable)
features_cat_labs = keep_existing(features_cat_labs, evaluable)
missing_indicator_cols = keep_existing(missing_indicator_cols, evaluable)

# Define two feature configurations
FEATURE_SETS = {
    "ClinicalOnly": {
        "num": features_num_base,
        "cat": features_cat_base,
        "extra_num": []  # no missing indicators here
    },
    "ClinicalPlusLabs_MissInd": {
        "num": features_num_base + features_num_labs + missing_indicator_cols,
        "cat": features_cat_base + features_cat_labs,
        "extra_num": []  # already included in num
    }
}

y = evaluable["EFS24_bin"].astype(int).values


# Helper: build pipeline and evaluate with CV
# -------------------------
def build_pipeline(num_cols, cat_cols):
    pre = ColumnTransformer(
        transformers=[
            ("num", Pipeline([
                ("imp", SimpleImputer(strategy="median")),
                ("sc", StandardScaler())
            ]), num_cols),
            ("cat", Pipeline([
                ("imp", SimpleImputer(strategy="most_frequent")),
                ("oh", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols),
        ],
        remainder="drop"
    )

    clf = LogisticRegression(
        penalty="elasticnet",
        solver="saga",
        l1_ratio=0.2,
        C=1.0,
        max_iter=8000
    )

    return Pipeline([("pre", pre), ("clf", clf)])

def cv_eval(X, y, pipeline, n_splits=5, seed=42):
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    aucs, aprs, briers = [], [], []
    oof = np.zeros_like(y, dtype=float)

    for fold, (tr, te) in enumerate(cv.split(X, y), start=1):
        pipeline.fit(X.iloc[tr], y[tr])
        p = pipeline.predict_proba(X.iloc[te])[:, 1]
        oof[te] = p
        aucs.append(roc_auc_score(y[te], p))
        aprs.append(average_precision_score(y[te], p))
        briers.append(brier_score_loss(y[te], p))
        print(f"  Fold {fold}: AUROC={aucs[-1]:.3f} | AUPRC={aprs[-1]:.3f} | Brier={briers[-1]:.3f}")

    return {
        "auroc_mean": float(np.mean(aucs)),
        "auroc_sd": float(np.std(aucs)),
        "auprc_mean": float(np.mean(aprs)),
        "auprc_sd": float(np.std(aprs)),
        "brier_mean": float(np.mean(briers)),
        "brier_sd": float(np.std(briers)),
        "oof_pred": oof
    }


# Run both baseline models
# -------------------------
results = {}

for name, cfg in FEATURE_SETS.items():
    num_cols = cfg["num"]
    cat_cols = cfg["cat"]

    print(f"\n=== Model: {name} ===")
    print("Numeric cols:", num_cols)
    print("Categorical cols:", cat_cols)

    X = evaluable[num_cols + cat_cols].copy()

    pipe = build_pipeline(num_cols, cat_cols)
    res = cv_eval(X, y, pipe, n_splits=5, seed=42)

    print(f"\n{name} summary:")
    print("  AUROC: mean {:.3f} (sd {:.3f})".format(res["auroc_mean"], res["auroc_sd"]))
    print("  AUPRC: mean {:.3f} (sd {:.3f})".format(res["auprc_mean"], res["auprc_sd"]))
    print("  Brier: mean {:.3f} (sd {:.3f})".format(res["brier_mean"], res["brier_sd"]))

    results[name] = res


=== Model: ClinicalOnly ===
Numeric cols: ['Age @ Diagnosis', 'FLIPI', 'Num of Ex. Sites']
Categorical cols: ['Gender', 'Race', 'Ann Arbor Stage', 'Ann Arbor Stage Group', 'PS Group', 'FLIPI Group', 'FL Grade', 'BM Involvement Group', 'Bulky Disease', 'B Symptoms']
  Fold 1: AUROC=0.523 | AUPRC=0.300 | Brier=0.206
  Fold 2: AUROC=0.604 | AUPRC=0.424 | Brier=0.190
  Fold 3: AUROC=0.587 | AUPRC=0.362 | Brier=0.196
  Fold 4: AUROC=0.693 | AUPRC=0.491 | Brier=0.182
  Fold 5: AUROC=0.557 | AUPRC=0.330 | Brier=0.201

ClinicalOnly summary:
  AUROC: mean 0.593 (sd 0.057)
  AUPRC: mean 0.381 (sd 0.069)
  Brier: mean 0.195 (sd 0.008)

=== Model: ClinicalPlusLabs_MissInd ===
Numeric cols: ['Age @ Diagnosis', 'FLIPI', 'Num of Ex. Sites', 'ANC', 'AMC', 'ALC', 'ANC__missing', 'AMC__missing', 'ALC__missing', 'LDH Group__missing']
Categorical cols: ['Gender', 'Race', 'Ann Arbor Stage', 'Ann Arbor Stage Group', 'PS Group', 'FLIPI Group', 'FL Grade', 'BM Involvement Group', 'Bulky Disease', 'B Symptoms

In [58]:
# Quick comparison
# -------------------------
print("\n=== Comparison ===")
for metric in ["auroc_mean", "auprc_mean", "brier_mean"]:
    print(metric,
          "ClinicalOnly:", results["ClinicalOnly"][metric],
          "| ClinicalPlusLabs_MissInd:", results["ClinicalPlusLabs_MissInd"][metric])


=== Comparison ===
auroc_mean ClinicalOnly: 0.5926474814998031 | ClinicalPlusLabs_MissInd: 0.6058922942646523
auprc_mean ClinicalOnly: 0.3813901953922725 | ClinicalPlusLabs_MissInd: 0.3933866815014491
brier_mean ClinicalOnly: 0.19497140601809054 | ClinicalPlusLabs_MissInd: 0.19414684202479476


# “Baseline clinical features demonstrated modest discrimination for early event-free survival failure. Inclusion of laboratory variables with explicit modeling of missingness yielded incremental improvements in discrimination and precision without compromising calibration.”

In [62]:
# Boosted tree baseline
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV

def keep_existing(cols):
    return [c for c in cols if c in evaluable.columns]

features_num_base = keep_existing(features_num_base)
features_cat_base = keep_existing(features_cat_base)
features_num_labs = keep_existing(features_num_labs)
features_cat_labs = keep_existing(features_cat_labs)
missing_indicator_cols = keep_existing(missing_indicator_cols)

FEATURE_SETS = {
    "ClinicalOnly": {
        "num": features_num_base,
        "cat": features_cat_base
    },
    "ClinicalPlusLabs_MissInd": {
        "num": features_num_base + features_num_labs + missing_indicator_cols,
        "cat": features_cat_base + features_cat_labs
    }
}

# Preprocessing + model
#    - Numeric: no imputation needed for HGB (it can handle NaNs),
#      but we DO want imputation for missing indicator cols? they are 0/1 already.
#      We'll pass numeric through as-is.
#    - Categorical: impute most frequent, one-hot encode.
# -------------------------
def make_pipeline(num_cols, cat_cols, calibrated=False):
    pre = ColumnTransformer(
        transformers=[
            ("num", "passthrough", num_cols),
            ("cat", Pipeline([
                ("imp", SimpleImputer(strategy="most_frequent")),
                ("oh", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols),
        ],
        remainder="drop"
    )

    hgb = HistGradientBoostingClassifier(
        loss="log_loss",
        learning_rate=0.05,
        max_depth=3,
        max_iter=500,
        min_samples_leaf=30,
        l2_regularization=0.0,
        random_state=42
    )

    if not calibrated:
        return Pipeline([("pre", pre), ("clf", hgb)])

    # Calibrate predicted probabilities (recommended for continuous risk reporting)
    # Using sigmoid for stability; isotonic can overfit in smaller folds.
    cal = CalibratedClassifierCV(estimator=hgb, method="sigmoid", cv=3)
    return Pipeline([("pre", pre), ("cal", cal)])

# Cross-validated evaluation (with out-of-fold predictions)
# -------------------------
def cv_eval(X, y, pipeline, n_splits=5, seed=42):
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    aucs, aprs, briers = [], [], []
    oof = np.zeros_like(y, dtype=float)

    for fold, (tr, te) in enumerate(cv.split(X, y), start=1):
        pipeline.fit(X.iloc[tr], y[tr])
        # calibrated pipeline returns predict_proba; uncalibrated also does
        p = pipeline.predict_proba(X.iloc[te])[:, 1]
        oof[te] = p
        aucs.append(roc_auc_score(y[te], p))
        aprs.append(average_precision_score(y[te], p))
        briers.append(brier_score_loss(y[te], p))
        print(f"  Fold {fold}: AUROC={aucs[-1]:.3f} | AUPRC={aprs[-1]:.3f} | Brier={briers[-1]:.3f}")

    return {
        "auroc_mean": float(np.mean(aucs)),
        "auroc_sd": float(np.std(aucs)),
        "auprc_mean": float(np.mean(aprs)),
        "auprc_sd": float(np.std(aprs)),
        "brier_mean": float(np.mean(briers)),
        "brier_sd": float(np.std(briers)),
        "oof_pred": oof
    }

# 4) Run: uncalibrated and calibrated boosted tree, for both feature sets
# -------------------------
results = {}

for set_name, cfg in FEATURE_SETS.items():
    num_cols, cat_cols = cfg["num"], cfg["cat"]
    X = evaluable[num_cols + cat_cols].copy()

    print(f"\n=== Boosted Tree (HGB) | {set_name} | Uncalibrated ===")
    pipe = make_pipeline(num_cols, cat_cols, calibrated=False)
    res_uncal = cv_eval(X, y, pipe, n_splits=5, seed=42)
    print(f"{set_name} HGB Uncal summary: AUROC {res_uncal['auroc_mean']:.3f} (sd {res_uncal['auroc_sd']:.3f}) | "
          f"AUPRC {res_uncal['auprc_mean']:.3f} | Brier {res_uncal['brier_mean']:.3f}")

    print(f"\n=== Boosted Tree (HGB) | {set_name} | Calibrated (sigmoid) ===")
    pipe_cal = make_pipeline(num_cols, cat_cols, calibrated=True)
    res_cal = cv_eval(X, y, pipe_cal, n_splits=5, seed=42)
    print(f"{set_name} HGB Cal summary:   AUROC {res_cal['auroc_mean']:.3f} (sd {res_cal['auroc_sd']:.3f}) | "
          f"AUPRC {res_cal['auprc_mean']:.3f} | Brier {res_cal['brier_mean']:.3f}")

    results[(set_name, "uncal")] = res_uncal
    results[(set_name, "cal")] = res_cal




=== Boosted Tree (HGB) | ClinicalOnly | Uncalibrated ===
  Fold 1: AUROC=0.538 | AUPRC=0.310 | Brier=0.215
  Fold 2: AUROC=0.596 | AUPRC=0.369 | Brier=0.196
  Fold 3: AUROC=0.526 | AUPRC=0.318 | Brier=0.214
  Fold 4: AUROC=0.659 | AUPRC=0.491 | Brier=0.181
  Fold 5: AUROC=0.564 | AUPRC=0.348 | Brier=0.204
ClinicalOnly HGB Uncal summary: AUROC 0.577 (sd 0.048) | AUPRC 0.367 | Brier 0.202

=== Boosted Tree (HGB) | ClinicalOnly | Calibrated (sigmoid) ===
  Fold 1: AUROC=0.542 | AUPRC=0.320 | Brier=0.198
  Fold 2: AUROC=0.618 | AUPRC=0.396 | Brier=0.193
  Fold 3: AUROC=0.525 | AUPRC=0.326 | Brier=0.199
  Fold 4: AUROC=0.661 | AUPRC=0.445 | Brier=0.195
  Fold 5: AUROC=0.580 | AUPRC=0.388 | Brier=0.194
ClinicalOnly HGB Cal summary:   AUROC 0.585 (sd 0.049) | AUPRC 0.375 | Brier 0.196

=== Boosted Tree (HGB) | ClinicalPlusLabs_MissInd | Uncalibrated ===
  Fold 1: AUROC=0.562 | AUPRC=0.347 | Brier=0.211
  Fold 2: AUROC=0.608 | AUPRC=0.391 | Brier=0.197
  Fold 3: AUROC=0.546 | AUPRC=0.334 | Br

In [65]:
# 5) Create risk groups from calibrated out-of-fold predictions (recommended)
#    Example: tertiles (low/intermediate/high) or top-quintile high risk.
# -------------------------
# Choose which model's predictions you want to use for risk grouping:
chosen = ("ClinicalPlusLabs_MissInd", "cal")
p = results[chosen]["oof_pred"]

tmp = evaluable.copy()
tmp["risk_prob"] = p

# Tertiles
tmp["risk_tertile"] = pd.qcut(tmp["risk_prob"], q=3, labels=["Low", "Intermediate", "High"])

# Top 20% high-risk flag (common operationalization)
thr_80 = np.quantile(tmp["risk_prob"], 0.80)
tmp["high_risk_top20"] = (tmp["risk_prob"] >= thr_80).astype(int)

print("\nRisk grouping summary (chosen model:", chosen, ")")
print(tmp.groupby("risk_tertile")["EFS24_bin"].mean())
print("Top 20% threshold:", thr_80)
print("Failure rate in top20%:", tmp.loc[tmp["high_risk_top20"]==1, "EFS24_bin"].mean())
print("Failure rate in bottom80%:", tmp.loc[tmp["high_risk_top20"]==0, "EFS24_bin"].mean())


Risk grouping summary (chosen model: ('ClinicalPlusLabs_MissInd', 'cal') )
risk_tertile
Low             0.225473
Intermediate    0.259897
High            0.333907
Name: EFS24_bin, dtype: float64
Top 20% threshold: 0.31097869149520746
Failure rate in top20%: 0.36962750716332377
Failure rate in bottom80%: 0.24892395982783358


/tmp/ipython-input-1010437205.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(tmp.groupby("risk_tertile")["EFS24_bin"].mean())
